In [39]:
import os
import librosa
import numpy as np
import soundfile as sf

In [40]:

def change_speed(audio, speed_factor=1.0):
    return librosa.effects.time_stretch(audio, rate=speed_factor)

In [41]:
def add_echo(audio, sr, delay=0.1, decay=0.5):
    delay_samples = int(sr * delay)
    echo_signal = np.zeros(len(audio) + delay_samples)
    echo_signal[:len(audio)] += audio
    echo_signal[delay_samples:] += decay * audio
    return echo_signal[:len(audio)]

In [42]:
def add_noise(audio, noise_factor=0.005):
    noise = np.random.randn(len(audio))
    augmented_audio = audio + noise_factor * noise
    return augmented_audio

In [43]:
def pitch_shift(audio, sr, n_steps=2):
    shifted_audio = librosa.effects.pitch_shift(audio, sr=sr, n_steps=n_steps)
    return shifted_audio

In [44]:
def split_audio(audio, sr, fragment_length=5):
    fragment_samples = int(sr * fragment_length)
    return [audio[i:i + fragment_samples] for i in range(0, len(audio), fragment_samples) if len(audio[i:i + fragment_samples]) == fragment_samples]

In [45]:
cur_dir = os.getcwd()
pd_folder = os.path.join(cur_dir, "../Data/ReadText/PD")
hc_folder = os.path.join(cur_dir, "../Data/ReadText/HC")
print(pd_folder)



d:\AndroidAppPark\Pipeline\PythonScripts\../Data/ReadText/PD


In [46]:

print(os.listdir(pd_folder))
print(os.listdir(hc_folder))

['ID02_pd_2_0_0 (mp3cut.net).wav', 'ID02_pd_2_0_0_S (mp3cut.net).wav', 'ID04_pd_2_0_1 (mp3cut.net).wav', 'ID04_pd_2_0_1_S (mp3cut.net).wav', 'ID06_pd_3_1_1 (mp3cut.net) (1).wav', 'ID06_pd_3_1_1_S (mp3cut.net).wav', 'ID07_pd_2_0_0 (mp3cut.net) (1).wav', 'ID07_pd_2_0_0_S (mp3cut.net).wav', 'ID13_pd_3_2_2 (mp3cut.net) (1).wav', 'ID13_pd_3_2_2_S (mp3cut.net).wav', 'ID16_pd_2_0_0 (mp3cut.net) (1).wav', 'ID16_pd_2_0_0_S (mp3cut.net).wav', 'ID17_pd_2_1_0 (mp3cut.net) (1).wav', 'ID17_pd_2_1_0_S (mp3cut.net) (1).wav', 'ID17_pd_2_1_0_S (mp3cut.net).wav', 'ID18_pd_4_3_3 (mp3cut.net) (1).wav', 'ID20_pd_3_0_1 (mp3cut.net) (1).wav', 'ID20_pd_3_0_1_S (mp3cut.net) (1).wav', 'ID20_pd_3_0_1_S (mp3cut.net).wav', 'ID24_pd_2_0_0 (mp3cut.net) (1).wav', 'ID24_pd_2_0_0_S (mp3cut.net).wav', 'ID27_pd_4_1_1 (mp3cut.net) (1).wav', 'ID27_pd_4_1_1_S (mp3cut.net) (1).wav', 'ID27_pd_4_1_1_S (mp3cut.net) (2).wav', 'ID27_pd_4_1_1_S (mp3cut.net).wav', 'ID29_pd_3_1_2 (mp3cut.net) (1).wav', 'ID29_pd_3_1_2_S (mp3cut.net).w

In [47]:

augmented_folder = "Augmented_Audio"
hc_augmented_folder = os.path.join(augmented_folder, "HC")
pd_augmented_folder = os.path.join(augmented_folder, "PD")

os.makedirs(hc_augmented_folder, exist_ok=True)
os.makedirs(pd_augmented_folder, exist_ok=True)

In [48]:

noise_factors = [0.001, 0.005, 0.01]
pitch_shifts = [-2, -1, 1, 2]
speed_factors = [0.9, 1.1]

In [49]:
for folder in [pd_folder, hc_folder]:
    for filename in os.listdir(folder):
        if filename.endswith(".wav"):
         
            audio_path = os.path.join(folder, filename)
            audio, sr = librosa.load(audio_path, sr=None)

           
            if folder == pd_folder:
                save_folder = pd_augmented_folder
            else:
                save_folder = hc_augmented_folder

            fragments = split_audio(audio, sr)

            for i, fragment in enumerate(fragments):
                fragment_filename = f"{filename}_fragment_{i}.wav"
                fragment_path = os.path.join(save_folder, fragment_filename)
                sf.write(fragment_path, fragment, sr)

                for noise_factor in noise_factors:
                    augmented_audio_with_noise = add_noise(fragment, noise_factor)
                    noise_filename = f"noise_{noise_factor}_{filename}_fragment_{i}.wav"
                    sf.write(os.path.join(save_folder, noise_filename), augmented_audio_with_noise, sr)

                for n_steps in pitch_shifts:
                    augmented_audio_with_pitch_shift = pitch_shift(fragment, sr, n_steps)
                    pitch_shift_filename = f"pitch_shift_{n_steps}_{filename}_fragment_{i}.wav"
                    sf.write(os.path.join(save_folder, pitch_shift_filename), augmented_audio_with_pitch_shift, sr)

                for speed_factor in speed_factors:
                    augmented_audio_with_speed_change = change_speed(fragment, speed_factor)
                    speed_change_filename = f"speed_{speed_factor}_{filename}_fragment_{i}.wav"
                    sf.write(os.path.join(save_folder, speed_change_filename), augmented_audio_with_speed_change, sr)

                augmented_audio_with_echo = add_echo(fragment, sr)
                echo_filename = f"echo_{filename}_fragment_{i}.wav"
                sf.write(os.path.join(save_folder, echo_filename), augmented_audio_with_echo, sr)

In [50]:
num_augmented_audio = len(os.listdir(hc_augmented_folder)) + len(os.listdir(pd_augmented_folder))

In [51]:
print("Количество аугментированных аудио:", num_augmented_audio)

Количество аугментированных аудио: 14707
